# Hi-C in HEK293T

reference:
http://www.pnas.org/content/111/3/996.long

dataset: 
1. https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1081530
2. https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1081531

downloaded files:
1. GSM1081530_CTRL_r1_cis.index.txt
2. GSM1081531_CTRL_r2_cis.index.txt

mapped to hg18
-> downloaded hg18.fa from phoenix: /local/data/iGenomes/Homo_sapiens/UCSC/hg18/Sequence/WholeGenomeFasta/genome.fa


gRNA file: (all the gRNAs are located on chrX)
HEK293T_gRNA sequence and deletion efficiency.xlsx


In [91]:
# Modules

# import regular expression module
import re

# import sys module
import sys

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import pandas as pd
import math

In [2]:
# function1 

# find positions on a defined chromosome, outputs the end position
def find_position (input_filename, genome_filename, chromosome):
	genome=list(SeqIO.parse(genome_filename, 'fasta'))
	seqs=list(SeqIO.parse(input_filename, 'fasta'))
	out_matrix=[]
	for k in range(len(seqs)):
		inp=seqs[k].seq
		inp=inp.upper()
		inp_rc=inp.reverse_complement()
		for i in range(len(genome)):
			if genome[i].id==chromosome:
				seq=genome[i].seq
				seq=seq.upper()
		pos1=seq.find(inp)+1
		end1=seq.find(inp)+len(inp)+1
		if pos1==0:
			pos2=seq.find(inp_rc)+1
			end2=seq.find(inp_rc)+len(inp_rc)+1
			out_matrix.append(seqs[k].description+'\t'+str(pos2)+'\t'+str(end2)+'\t'+'-')
		else:
			out_matrix.append(seqs[k].description+'\t'+str(pos1)+'\t'+str(end1)+'\t'+'+')
	f=open(input_filename+'.txt','w+')
	f.write('\n'.join(i for i in out_matrix))
	f.close()


In [33]:
# creates a fasta file from HEK293T_gRNA sequence and deletion efficiency.xlsx
grna=pd.read_excel('../../../../../Volumes/My_Book_for_Mac/hic/hek293t/HEK293T_gRNA sequence and deletion efficiency_chrX.xlsx',sheet_name=0)


In [34]:
grna

,gRNA,Sequence,Deletion efficiency (%)
0,IN1,GGTTTCAATGAGAGCATTAC,NaN
1,IN2,GTGGAAGTTTAATGACTAAG,3.3
2,IN3,GTACTTGCTTTCATTTCACT,3.3
3,IN4,GAAATGGGTTATTTAACCCT,10.0
4,50K,GTTAATTTGCACTTTGATGC,10.0
5,100K,GAACTTTCTCAACCTCATAA,10.0
6,500K,GCTCAGCATGAATTTATCTT,10.0
7,1000K,GCAGTGCCTAATTGCATGCT,1.0


In [35]:
out=[]

for i in range(grna.shape[0]):
    out.append('>'+grna['gRNA'][i]+'\n'+str(grna['Sequence'][i][1:21]))

In [36]:
out

['>IN1\nGTTTCAATGAGAGCATTAC',
 '>IN2\nTGGAAGTTTAATGACTAAG',
 '>IN3\nTACTTGCTTTCATTTCACT',
 '>IN4\nAAATGGGTTATTTAACCCT',
 '>50K\nTTAATTTGCACTTTGATGC',
 '>100K\nAACTTTCTCAACCTCATAA',
 '>500K\nCTCAGCATGAATTTATCTT',
 '>1000K\nCAGTGCCTAATTGCATGCT']

In [37]:
f=open('../../../../../Volumes/My_Book_for_Mac/hic/hek293t/hek293t_grna.fa','w')
f.write('\n'.join(i for i in out))
f.close()


In [38]:
input_filename='../../../../../Volumes/My_Book_for_Mac/hic/hek293t/hek293t_grna.fa'
genome_filename='../../../../../Volumes/My_Book_for_Mac/hic/hek293t/hg18.fa'


In [39]:
find_position (input_filename, genome_filename, 'chrX')


# load in the Hi-C data

In [40]:
hic_filename='../../../../../Volumes/My_Book_for_Mac/hic/hek293t/GSM1081530_CTRL_r1_cis.index.txt'

In [41]:
#data=[]
#with open(hic_filename) as fp:
#    for i, line in enumerate(fp):
#        if line.split('\t')[0]=='chrX':
#            data.append(line)
#fp.close()

In [46]:
data=pd.read_csv(hic_filename,sep='\t',header=None)

In [54]:
data_chrx=data[data[0]=='chrX']

In [55]:
data_chrx.head()

,0,1,2,3
15368961,chrX,2680000_2720000,2720000_2760000,23.898
15368962,chrX,2680000_2720000,2760000_2800000,12.508
15368963,chrX,2680000_2720000,2800000_2840000,0.000
15368964,chrX,2680000_2720000,2840000_2880000,5.680
15368965,chrX,2680000_2720000,2880000_2920000,0.000


In [68]:
data2=data_chrx[[0]]
region1=data_chrx[1].str.split('_', 1, expand=True)
region1.columns=['region1_start','region1_end']
region2=data_chrx[2].str.split('_', 1, expand=True)
region2.columns=['region2_start','region2_end']

In [73]:
region1.head()

,region1_start,region1_end
15368961,2680000,2720000
15368962,2680000,2720000
15368963,2680000,2720000
15368964,2680000,2720000
15368965,2680000,2720000


In [74]:
region2.head()

,region2_start,region2_end
15368961,2720000,2760000
15368962,2760000,2800000
15368963,2800000,2840000
15368964,2840000,2880000
15368965,2880000,2920000


In [77]:
combine=region1.join(region2)
combine=combine.join(data_chrx[[3]])

In [113]:
combine.columns=['region1_start','region1_end','region2_start','region2_end','score']
combine['region1_start']=combine['region1_start'].astype('int')
combine['region1_end']=combine['region1_end'].astype('int')
combine['region2_start']=combine['region2_start'].astype('int')
combine['region2_end']=combine['region2_end'].astype('int')

In [115]:
combine.dtypes

region1_start      int64
region1_end        int64
region2_start      int64
region2_end        int64
score            float64
dtype: object

In [114]:
combine.head()

,region1_start,region1_end,region2_start,region2_end,score
15368961,2680000,2720000,2720000,2760000,23.898
15368962,2680000,2720000,2760000,2800000,12.508
15368963,2680000,2720000,2800000,2840000,0.000
15368964,2680000,2720000,2840000,2880000,5.680
15368965,2680000,2720000,2880000,2920000,0.000


In [81]:
grna_loc=pd.read_csv('../../../../../Volumes/My_Book_for_Mac/hic/hek293t/hek293t_grna.fa.txt',sep='\t',header=None)

In [82]:
grna_loc

,0,1,2,3
0,IN1,133435017,133435036,-
1,IN2,133436794,133436813,+
2,IN3,133437139,133437158,+
3,IN4,133448330,133448349,+
4,50K,133372126,133372145,+
5,100K,133322105,133322124,+
6,500K,132921439,132921458,-
7,1000K,132417203,132417222,+


In [120]:
fl=[]
ce=[]
for i in grna_loc[2]:
    fl.append(math.floor(i/40000)*40000)
    ce.append(math.ceil(i/40000)*40000)

grna_loc['range1']=fl
grna_loc['range2']=ce

In [126]:
grna_loc

,0,1,2,3,range1,range2
0,IN1,133435017,133435036,-,133400000,133440000
1,IN2,133436794,133436813,+,133400000,133440000
2,IN3,133437139,133437158,+,133400000,133440000
3,IN4,133448330,133448349,+,133440000,133480000
4,50K,133372126,133372145,+,133360000,133400000
5,100K,133322105,133322124,+,133320000,133360000
6,500K,132921439,132921458,-,132920000,132960000
7,1000K,132417203,132417222,+,132400000,132440000


In [116]:
anchor_right=combine[combine['region1_start']==grna_loc['range1'][0]]

In [117]:
anchor_right.head()

,region1_start,region1_end,region2_start,region2_end,score
16132071,133400000,133440000,133440000,133480000,9.935
16132072,133400000,133440000,133480000,133520000,5.426
16132073,133400000,133440000,133520000,133560000,2.183
16132074,133400000,133440000,133560000,133600000,4.584
16132075,133400000,133440000,133600000,133640000,4.908


In [118]:
anchor_left=combine[combine['region2_start']==grna_loc['range1'][0]]

In [129]:
anchor_left.tail()

,region1_start,region1_end,region2_start,region2_end,score
16130845,133200000,133240000,133400000,133440000,2.569
16131090,133240000,133280000,133400000,133440000,2.608
16131335,133280000,133320000,133400000,133440000,3.679
16131580,133320000,133360000,133400000,133440000,3.866
16131825,133360000,133400000,133400000,133440000,9.935


In [175]:
grna_loc2=grna_loc.drop([0],axis=0)
scores=[]
for index, g in grna_loc2.iterrows():
    #print (index)
    if g['range1']==grna_loc['range1'][0]:
        scores.append(10)
    elif g['range1'] > grna_loc['range1'][0]:
        #print('>')
        #print(anchor_right[anchor_right['region2_start']==g['range1']])
        tmp=float(anchor_right[anchor_right['region2_start']==g['range1']]['score'])
        scores.append(tmp)
    elif g['range1'] < grna_loc['range1'][0]:
        #print('<')
        tmp=float(anchor_left[anchor_left['region1_start']==g['range1']]['score'])
        scores.append(tmp)

In [176]:
scores

[10, 10, 9.935, 9.935, 3.866, 1.234, 1.151]

In [177]:
grna_loc2['score']=scores

In [178]:
grna_loc2

,0,1,2,3,range1,range2,score
1,IN2,133436794,133436813,+,133400000,133440000,10.000
2,IN3,133437139,133437158,+,133400000,133440000,10.000
3,IN4,133448330,133448349,+,133440000,133480000,9.935
4,50K,133372126,133372145,+,133360000,133400000,9.935
5,100K,133322105,133322124,+,133320000,133360000,3.866
6,500K,132921439,132921458,-,132920000,132960000,1.234
7,1000K,132417203,132417222,+,132400000,132440000,1.151


In [179]:
grna_loc2['efficiency']=grna['Deletion efficiency (%)'][1:]

In [180]:
grna_loc2

,0,1,2,3,range1,range2,score,efficiency
1,IN2,133436794,133436813,+,133400000,133440000,10.000,3.3
2,IN3,133437139,133437158,+,133400000,133440000,10.000,3.3
3,IN4,133448330,133448349,+,133440000,133480000,9.935,10.0
4,50K,133372126,133372145,+,133360000,133400000,9.935,10.0
5,100K,133322105,133322124,+,133320000,133360000,3.866,10.0
6,500K,132921439,132921458,-,132920000,132960000,1.234,10.0
7,1000K,132417203,132417222,+,132400000,132440000,1.151,1.0


In [181]:
grna_loc2.to_csv('../../../../../Volumes/My_Book_for_Mac/hic/hek293t/hek293t_result.txt',sep='\t')

# running HiCPlotter

# convert into contact matrix
python bedToMatrix.py GSM1081530_CTRL_r1_cis.index_chrX.txt

# plot a specific region
python HiCPlotter.py -f GSM1081530_CTRL_r1_cis.index_chrX.txt_matrix.txt -chr chrX -n hek293t -o example -r 40000 -s 3275 -e 3375

# plot with TADs and peaks
python HiCPlotter.py -f GSM1081530_CTRL_r1_cis.index_chrX.txt_matrix.txt -chr chrX -n hek293t -o example -r 40000 -s 3275 -e 3375 -hmc 1 -ptd 1 -peak peaks.bedgraph

# plot with arcs
python HiCPlotter.py -f GSM1081530_CTRL_r1_cis.index_chrX.txt_matrix.txt -chr chrX -n hek293t -o example -r 40000 -s 3275 -e 3375 -hmc 1 -ptd 1 -a arcs.bedgraph -al gRNAs -da 1 -mm 6 -spi 1 -ac cecece

python HiCPlotter.py -f GSM1081530_CTRL_r1_cis.index_chrX.txt_matrix.txt -chr chrX -n hek293t -o example -r 40000 -s 3275 -e 3375 -hmc 1 -ptd 1 -a arcs.bedgraph -al gRNAs -da 1 -mm 6 -spi 1 -ac cecece

